In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install datasets transformers

In [3]:
!pip install datasets --upgrade

In [5]:
!pip install evaluate


In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EvalPrediction
from datasets import load_dataset
from sklearn.metrics import roc_auc_score
import numpy as np
import matplotlib.pyplot as plt

In [7]:
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    TrainingArguments, 
    Trainer, 
    EvalPrediction, 
    TrainerCallback
)
from datasets import load_dataset
from sklearn.metrics import roc_auc_score, accuracy_score, precision_recall_fscore_support
import numpy as np
import matplotlib.pyplot as plt

In [8]:
# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [9]:
# Load the MNLI dataset
dataset = load_dataset("glue", "mnli")

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/52.2M [00:00<?, ?B/s]

(…)alidation_matched-00000-of-00001.parquet:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

(…)dation_mismatched-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

test_matched-00000-of-00001.parquet:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

In [10]:
# Load LastBERT tokenizer and model for classification task (MNLI)
tokenizer = AutoTokenizer.from_pretrained("Peraboom/LastBERT")
model = AutoModelForSequenceClassification.from_pretrained(
    "Peraboom/LastBERT", num_labels=3, ignore_mismatched_sizes=True
).to(device)  # 3 labels for MNLI (entailment, contradiction, neutral)

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/119M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Peraboom/LastBERT and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([3]) in the model instantiated
- classifier.weight: found shape torch.Size([2, 384]) in the checkpoint and torch.Size([3, 384]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], padding="max_length", truncation=True)

# Tokenize the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

Map:   0%|          | 0/9832 [00:00<?, ? examples/s]

Map:   0%|          | 0/9796 [00:00<?, ? examples/s]

Map:   0%|          | 0/9847 [00:00<?, ? examples/s]

In [15]:
# Define Trainer Callback for Metrics Logging
class MetricsCallback(TrainerCallback):
    def __init__(self):
        self.metrics = {"train_loss": [], "eval_loss": [], "accuracy": [], "precision": [], "recall": [], "f1": []}

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            if "loss" in logs:
                self.metrics["train_loss"].append(logs["loss"])
            if "eval_loss" in logs:
                self.metrics["eval_loss"].append(logs["eval_loss"])
            if "eval_accuracy" in logs:
                self.metrics["accuracy"].append(logs["eval_accuracy"])
            if "eval_precision" in logs:
                self.metrics["precision"].append(logs["eval_precision"])
            if "eval_recall" in logs:
                self.metrics["recall"].append(logs["eval_recall"])
            if "eval_f1" in logs:
                self.metrics["f1"].append(logs["eval_f1"])

# Create an instance of the MetricsCallback
metrics_callback = MetricsCallback()

# Training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    save_strategy="epoch",  # Match this with evaluation_strategy
    logging_dir='./logs_mnli',
    output_dir='./results_mnli',
    load_best_model_at_end=True,
)

# Compute metrics function
def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids

    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    
    # Convert logits to probabilities
    probabilities = torch.softmax(torch.tensor(p.predictions), dim=1).numpy()

    # Calculate ROC AUC for multi-class
    roc_auc = roc_auc_score(labels, probabilities, multi_class='ovr')
    
    return {
        "roc_auc": roc_auc,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

# Set up the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation_matched"],
    compute_metrics=compute_metrics,
    callbacks=[metrics_callback],  # Add the metrics callback here
)

# Train and evaluate the model
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Roc Auc,Accuracy,Precision,Recall,F1
1,0.735300,0.779045,0.847707,0.674885,0.679468,0.674885,0.675266
2,0.681500,0.718495,0.864260,0.697606,0.696272,0.697606,0.696152
3,0.614600,0.732800,0.868526,0.707489,0.707875,0.707489,0.707658


TrainOutput(global_step=147264, training_loss=0.7023609448183832, metrics={'train_runtime': 17854.9016, 'train_samples_per_second': 65.982, 'train_steps_per_second': 8.248, 'total_flos': 6.472346586625843e+16, 'train_loss': 0.7023609448183832, 'epoch': 3.0})

In [16]:
# Optionally, plot the metrics
def plot_metrics(metrics):
    plt.figure(figsize=(12, 6))
    plt.plot(metrics['eval_accuracy'], label="Accuracy", marker='o')
    plt.plot(metrics['eval_precision'], label="Precision", marker='o')
    plt.plot(metrics['eval_recall'], label="Recall", marker='o')
    plt.plot(metrics['eval_f1'], label="F1 Score", marker='o')
    plt.xlabel("Epochs")
    plt.ylabel("Score")
    plt.title("Evaluation Metrics Over Epochs")
    plt.legend()
    plt.grid(True)
    plt.show()

# Call this function if you want to visualize the metrics over epochs
plot_metrics(metrics_callback.metrics)  # Uncomment this line to plot metrics

KeyError: 'eval_accuracy'

<Figure size 1200x600 with 0 Axes>

***7 epoch***

In [13]:
# Define Trainer Callback for Metrics Logging
class MetricsCallback(TrainerCallback):
    def __init__(self):
        self.metrics = {"train_loss": [], "eval_loss": [], "accuracy": [], "precision": [], "recall": [], "f1": []}

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            if "loss" in logs:
                self.metrics["train_loss"].append(logs["loss"])
            if "eval_loss" in logs:
                self.metrics["eval_loss"].append(logs["eval_loss"])
            if "eval_accuracy" in logs:
                self.metrics["accuracy"].append(logs["eval_accuracy"])
            if "eval_precision" in logs:
                self.metrics["precision"].append(logs["eval_precision"])
            if "eval_recall" in logs:
                self.metrics["recall"].append(logs["eval_recall"])
            if "eval_f1" in logs:
                self.metrics["f1"].append(logs["eval_f1"])

# Create an instance of the MetricsCallback
metrics_callback = MetricsCallback()

# Training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=7,
    evaluation_strategy="epoch",
    save_strategy="epoch",  # Match this with evaluation_strategy
    logging_dir='./logs_mnli',
    output_dir='./results_mnli',
    load_best_model_at_end=True,
)

# Compute metrics function
def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids

    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    
    # Convert logits to probabilities
    probabilities = torch.softmax(torch.tensor(p.predictions), dim=1).numpy()

    # Calculate ROC AUC for multi-class
    roc_auc = roc_auc_score(labels, probabilities, multi_class='ovr')
    
    return {
        "roc_auc": roc_auc,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

# Set up the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation_matched"],
    compute_metrics=compute_metrics,
    callbacks=[metrics_callback],  # Add the metrics callback here
)

# Train and evaluate the model
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Roc Auc,Accuracy,Precision,Recall,F1
1,0.827500,0.844819,0.817397,0.630565,0.651228,0.630565,0.630745
2,0.759200,0.758666,0.842955,0.672134,0.671098,0.672134,0.671231
3,0.688400,0.753807,0.857879,0.682119,0.706037,0.682119,0.682923
4,0.688700,0.752858,0.861179,0.694244,0.698445,0.694244,0.695331
5,0.607900,0.752539,0.865170,0.698523,0.697868,0.698523,0.697288
6,0.567500,0.774891,0.866754,0.705451,0.706069,0.705451,0.705647
7,0.519200,0.812184,0.865524,0.705858,0.706644,0.705858,0.706178


TrainOutput(global_step=343616, training_loss=0.6794921848600327, metrics={'train_runtime': 41696.707, 'train_samples_per_second': 65.926, 'train_steps_per_second': 8.241, 'total_flos': 1.51021420354603e+17, 'train_loss': 0.6794921848600327, 'epoch': 7.0})